In [6]:
import json 
import numpy as np
import faiss
import os

# Path file
embd_path = r"D:/chatbot_covid19_intern_procodecg/vector_db/faiss__metadata.json"
faiss_path = r"D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"

# Pastikan folder tujuan ada
os.makedirs(os.path.dirname(faiss_path), exist_ok=True)

# List untuk menampung data
text = []
vectors = []

# Membaca file embedding
with open(embd_path, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        text.append(record["text"])
        vectors.append(record["embedding"])

# Ubah list ke numpy array
vectors = np.array(vectors).astype("float32")

# Buat FAISS index (L2 distance)
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

# Simpan FAISS index ke file .index
faiss.write_index(index, faiss_path)

# Info hasil
print(f"💚 FAISS index disimpan di : {faiss_path}")
print(f"Total data: {len(text)} chunk")

# Simpan teks referensi dalam file JSON (bukan FAISS)
text_json_path = faiss_path.replace(".index", "_texts.json")
with open(text_json_path, "w", encoding="utf-8") as f:
    json.dump(text, f, ensure_ascii=False, indent=2)

print(f"📝 File teks disimpan di : {text_json_path}")


💚 FAISS index disimpan di : D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index
Total data: 20 chunk
📝 File teks disimpan di : D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19_texts.json


In [7]:
import os

faiss_path = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"
print("Ada file?:", os.path.exists(faiss_path))
print("Ukuran file (byte):", os.path.getsize(faiss_path) if os.path.exists(faiss_path) else 0)


Ada file?: True
Ukuran file (byte): 61485


In [3]:
with open("D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index", "rb") as f:
    header = f.read(20)
print(header)


b'IxF2\x00\x03\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00'


In [5]:
import faiss

# --- 1. Muat (Load) Indeks FAISS ---
# Ganti path sesuai dengan lokasi file Anda jika perlu
index = faiss.read_index("D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index")

# --- 2. Cek Dimensi Vektor (d) ---
dimensi_faiss = index.d

print(f"✅ Dimensi Vektor (index.d) dalam file FAISS: {dimensi_faiss}")

✅ Dimensi Vektor (index.d) dalam file FAISS: 768
